In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row

import os
import sys
from pyspark.sql.types import *

from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from numpy import array

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1553734074701_0006,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
# 0 Date
# 1 Time
# 2 TargetTemp
# 3 ActualTemp
# 4 System
# 5 SystemAge
# 6 BuildingID

LabeledDocument = Row("BuildingID", "SystemInfo", "label")

# Define a function that parses the raw CSV file and returns an object of type LabeledDocument

def parseDocument(line):
    values = [str(x) for x in line.split(',')]
    if (values[3] > values[2]):
        hot = 1.0
    else:
        hot = 0.0        

    textValue = str(values[4]) + " " + str(values[5])

    return LabeledDocument((values[6]), textValue, hot)

 # Load the raw HVAC.csv file, parse it using the function
data = sc.textFile("wasbs:///HdiSamples/HdiSamples/SensorSampleData/hvac/HVAC.csv")

documents = data.filter(lambda s: "Date" not in s).map(parseDocument)
training = documents.toDF()

In [3]:
tokenizer = Tokenizer(inputCol="SystemInfo", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to the training document
model = pipeline.fit(training)

In [4]:
training.show()

+----------+----------+-----+
|BuildingID|SystemInfo|label|
+----------+----------+-----+
|         4|     13 20|  0.0|
|        17|      3 20|  0.0|
|        18|     17 20|  1.0|
|        15|      2 23|  0.0|
|         3|      16 9|  1.0|
|         4|     13 28|  0.0|
|         2|     12 24|  0.0|
|        16|     20 26|  1.0|
|         9|      16 9|  1.0|
|        12|       6 5|  0.0|
|        15|     10 17|  1.0|
|         7|      2 11|  0.0|
|        15|      14 2|  1.0|
|         6|       3 2|  0.0|
|        20|     19 22|  0.0|
|         8|     19 11|  0.0|
|         6|      15 7|  0.0|
|        13|      12 5|  0.0|
|         4|      8 22|  0.0|
|         7|      17 5|  0.0|
+----------+----------+-----+
only showing top 20 rows

In [7]:
Document = Row("Id", "SystemInfo")
test=sc.parallelize([(1L, "20 25"),(2L, "4 15"),(3L, "16 9"),(4L,"9 22"),(5L,"17 10"),(6L,"7 22")])\.map(lambda x: Document(*x)).toDF()

invalid syntax (<stdin>, line 2)
  File "<stdin>", line 2
    test=sc.parallelize([(1L, "20 25"),(2L, "4 15"),(3L, "16 9"),(4L,"9 22"),(5L,"17 10"),(6L,"7 22")])\.map(lambda x: Document(*x)).toDF()
                           ^
SyntaxError: invalid syntax



In [8]:
prediction=model.transform(test)
selected=prediction.select("SystemInfo","prediction","probability")
for row in selected.collect():
    print row

Missing parentheses in call to 'print' (<stdin>, line 4)
  File "<stdin>", line 4
    print row
            ^
SyntaxError: Missing parentheses in call to 'print'

